In [ ]:
#Installing webdriver-manager when running code for the first time
#!pip install webdriver-manager

In [7]:
#Importing Required Packages

#Importing numpy to handle arrays
import numpy as np
#Importing pandas to handle datasets
import pandas as pd

#Importing required Selenium Packages
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
#Check if Item is a Webelement
from selenium.webdriver.remote.webelement import WebElement
# Importing ElementClickInterceptedException from selenium.common.exceptions
# to be able to interpret 
from selenium.common.exceptions import ElementClickInterceptedException

#Import time to wait a number of seconds between selenium actions
import time



In [8]:
#Defining Driver Path
driver = webdriver.Chrome(ChromeDriverManager().install())

#Directing Driver to MPT Timetables page 
driver.get('https://www.publictransport.com.mt/en/timetables')

#Obtaining Selenium elements of all Route Numbers and Route Names in MPT Timetable page
Route_Numbers = driver.find_elements(By.XPATH, '/html/body/form/div[3]/section[2]/div[2]/div/div/div/div/div/ul/li/a/b')
Route_Names = driver.find_elements(By.XPATH, '/html/body/form/div[3]/section[2]/div[2]/div/div/div/div/div/ul/li/a/span')

#Obtaining Text for each Selenium Element (for loop over all Selenium Elements defined in Route_Numbers and Route_Names)
# List_Route_Numbers - A list of all Route Numbers in Malta & Gozo
# List_Route_Names - A list of all Route Names in Malta & Gozo
#Obtaining Text for each Selenium Object
List_Route_Numbers = []
List_Route_Names = []

for i in range(len(Route_Numbers)):
    List_Route_Numbers.append(Route_Numbers[i].text)
    List_Route_Names.append(Route_Names[i].text)

#Creating a Dictionary consisting of Route_Numbers and Route_Names
dict = {'Route Number': List_Route_Numbers, 'Route Names': List_Route_Names}
#Creating Dataframe consisting of two columns (Route Number and corresponding Route Names)
Route_Info = pd.DataFrame(dict)

#Saving Route_Info Dataframe
Route_Info.to_csv('C:\\Users\\Owner\\ICT5012 - Disseration\\Preliminary_Route_Info.csv', index=False)  

In [9]:
#Obtaining a list of Selenium elements of all Routes
Route_Stops = driver.find_elements(By.XPATH, '/html/body/form/div[3]/section[2]/div[2]/div/div/div/div/div/ul/li/a')

#Going over all Routes defined in Route_Info and obtain all Stops and their corresponding time

#Step 1 - Creating An empty dataframe to be used to store final result

# Create a list of columns labeled Stop Time X (X ranges from 1 to 50)
Stop_Time_Columns = [f"Stop Time {i}" for i in range(1, 51)]
# Defining all Columns to be used in final datafram (Initial list of columns & Stop_Time_Columns)
Columns = ['Route Direction', 'Stops', 'City Name', 'Date'] + Stop_Time_Columns
# Final Dataframe
Final_Dataframe = pd.DataFrame(columns=Columns)


#Step 2 - For loop going over list of Route_Stops, entering Route_Stops and obtain all Bus Stops
# and corresponding time in Route

# for loop going over all Routes defined in Route_Stops
for i in range(len(Route_Stops)):
    try: 
        
        #Part 1 - Entering ith Route 
        
        # Click on the ith Route
        Route_Stops[i].click()
        # Wait two seconds before proceeding to the next Selenium action
        time.sleep(2)
        # Obtaining Name of Route (Route XXX)
        Route_Name = driver.find_element(By.XPATH,'/html/body/form/div[3]/section[2]/div/div/div[1]/h1').text
        # Print Current Route Name to have an idea how for loop is progressing
        print(Route_Name)
        
        #Part 2 - Expaning to Obtain All Stops and all times

        # Expanding to See times only for stops (times for specific cities are hidden)
        
        # Removing Times for all cities (Done by changing Class to 'main-row first opened')
        Expand_All_Button1_XPATH = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[not(position()=1 or position()=2)]/div/table/tbody/tr[2]/td/div/table/tbody/tr[1]'
        ShowStops_Button1 = driver.find_elements(By.XPATH,Expand_All_Button1_XPATH)
        New_Class = 'main-row first opened'
        for i in range(len(ShowStops_Button1)):
            driver.execute_script("arguments[0].setAttribute('class', arguments[1]);", ShowStops_Button1[i], New_Class)

        # Obtaining Times for only Stops (Done by changing Class to 'main-row first opened')
        Expand_All_Button2_XPATH = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[not(position()=1 or position()=2)]/div/table/tbody/tr[2]/td/div/table/tbody/tr[not(position()=1)]'
        ShowStops_Button2 = driver.find_elements(By.XPATH,Expand_All_Button2_XPATH)
        New_Class = 'main-row additional opened'
        for i in range(len(ShowStops_Button2)):
            driver.execute_script("arguments[0].setAttribute('class', arguments[1]);", ShowStops_Button2[i], New_Class)

        # Show all times available for each stop throughout the entire day
        
        # Obtain all possible times (Done by changing class to 'schedule-part visible')
        Show_All_Time = driver.find_elements_by_class_name('schedule-part')
        new_class = "schedule-part visible"
        for i in range(len(Show_All_Time)):
            driver.execute_script("arguments[0].setAttribute('class', arguments[1]);", Show_All_Time[i], new_class)

        #Part 3 - Finding Route Directions
        Route_Direction = driver.find_elements(By.XPATH,'/html/body/form/div[3]/section[2]/div/div/div[2]/div/div/table/thead/tr/th')

        #Part 4 - for loop going over all different routes in Route_Direction list 
        # This is followed for loop going over all dates in the specific Route_Direction
        
        #first loop going over all different routes in Route_Direction list
        for A in range(len(Route_Direction)):
            
            DataFrames_Time = []
            
            # Obtaining Selenium Elements of all Dates in Ath Route Direction
            Dates_XPATH = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[1]/td[1]/select/option'
            Dates = driver.find_elements(By.XPATH,Dates_XPATH)
            
            
            #Second for loop going over all different dates in Ath Route_Direction 
            for C in range(len(Dates)):

                #Creating empty list to hold dataframes for Cities, Stops and corresponding Stops_Time
                Cities = []
                Stops = []
                Stops_Time = []

                #Click on C date 
                Dates[C].click()
                
                # Part 4.1 -  Calculate Number of Stop Times Per Stop
                #First Stop is used since all stops should have the same number of stop times
                Stop_Length_Xpath = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[2]/td/div[{C+1}]/table[1]/tbody/tr[1]/td/span'
                No_Stops_Per_Stop = driver.find_elements(By.XPATH,Stop_Length_Xpath)
                Length = len(No_Stops_Per_Stop)-1
                    
                
                # Part 4.2 - Collect All Cities in which bus stops are located  
                City_Xpath = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[2]/td/div[{C+1}]/table/tbody/tr[1]/td[1]/span'
                City_Name = driver.find_elements(By.XPATH, City_Xpath)
                for i in range(len(City_Name)):
                    Cities.append(City_Name[i].text)
                Cities = [item for item in Cities if item]
                
                # Part 4.3 - Collect All Bus Stop Locations
                Stop_Xpath = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[2]/td/div[{C+1}]/table/tbody/tr[not(position()=1)]/td[1]/span'
                Stop_Name = driver.find_elements(By.XPATH, Stop_Xpath)
                for i in range(len(Stop_Name)):
                    Stops.append(Stop_Name[i].text)
                Stops = [item for item in Stops if item]
                
                # Part 4.4 - Obtain Stop Times of All Bus Stop Location 
                # NB: Stop Times will also include Cities and Stops names  
                # All found in Cities and Stops list
                Times_Xpath = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[2]/td/div[{C+1}]/table/tbody/tr/td/span'
                Stop_Times = driver.find_elements(By.XPATH,Times_Xpath)
                for i in range(len(Stop_Times)):
                    Stops_Time.append(Stop_Times[i].text)
                Stops_Time = [item for item in Stops_Time if item]

                # Part 4.5 - Remove Cities and Stops from Stops_Time
                Stops_Time = [item for item in Stops_Time if item not in Cities]
                Stops_Time = [item for item in Stops_Time if item not in Stops]

                # Part 4.6 - Creating DataFrame consisting of one column for each time entry in a stop
                
                # Create a dictionary with L lists named 'Stop Time 1', 'Stop Time 2', etc.
                Stop_Times_Predf = {f'Stop Time {i + 1}': [] for i in range(Length)}
                # Distribute the items in the Stops_Time into Stop_Times_Predf
                for i, item in enumerate(Stops_Time):  # Assuming A is the list of items
                    Stop_Times_Predf[f'Stop Time {(i % Length) + 1}'].append(item)
                # Creating Dataframe using Stop_Times_Predf
                df = pd.DataFrame(Stop_Times_Predf)

                # Part 4.7 - Input Route Date, Stops and City Name into df
                
                # Insert Current Date to df
                df.insert(0, 'Date', Dates[C].text)
                # Insert list of Stops to df
                df.insert(0,'Stops', Stops)

                # Creating a Dataframe consisting of list of stops and their respective cities
                # Similar to vlookup in Excel, each stop will then be labbelled with a corresponding city
                
                #Obtain a list of selenium objects representing all cities and stops
                City_Stop_Xpath = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[2]/td/div[{C+1}]/table/tbody/tr/td[1]/span'
                City_Stop = driver.find_elements(By.XPATH, City_Stop_Xpath)
                City_Stop_List = []
                for i in range(len(City_Stop)):
                    City_Stop_List.append(City_Stop[i].text)
                
                data = []
                current_b_item = None
                #for loop going over list of City and Stops and if a city is found (done by comparing
                #City_Stop_List to Cities list it is appended to a list called current_b_item)
                for item in City_Stop_List:
                    if item in Cities:
                        current_b_item = item
                    else:
                        if current_b_item:
                            data.append([item, current_b_item])
                            
                # Creating City_Stop_DataFrame consisting of Stops and City Name Column
                City_Stop_DataFrame = pd.DataFrame(data, columns=['Stops', 'City Name'])
                # Remove duplicates from City_Stop_DataFrame to ensure 
                City_Stop_DataFrame.drop_duplicates(inplace = True)
                # Adding index to df to ensure it is not shuffled after carrying out merge function 
                df['original_index'] = df.index
                # Ensure both 'Stops' columns in df and City_Stop_DataFrame are strings
                City_Stop_DataFrame['Stops'] = City_Stop_DataFrame['Stops'].astype(str)
                df['Stops'] = df['Stops'].astype(str)
                # Using pd.merge to carry out operation similar to VLOOKUP-like operation
                DataFrame_OneTime = pd.merge(City_Stop_DataFrame, df, on='Stops', how='left')
                # Sort back to the original order using the index and re-name dataframe to DataFrame_OneTime
                DataFrame_OneTime = DataFrame_OneTime.sort_values('original_index').drop('original_index', axis=1)
                
                #Part 4.8 - Append DataFrame_OneTime to DataFrames_Time
                DataFrames_Time.append(DataFrame_OneTime)
                
                #Part 4.9 - Append Dataframe for all Times
                OneRoute_AllTimes_DataFrame = pd.concat(DataFrames_Time, ignore_index=True)

            # Part 4.10 - Append Ath Route_Direction to OneRoute_AllTimes_DataFrame and concatenate to 
            # Final_Dataframe which will compile all Dataframes
            OneRoute_AllTimes_DataFrame.insert(0,'Route Direction',Route_Direction[A].text)
            Final_Dataframe = pd.concat([Final_Dataframe, OneRoute_AllTimes_DataFrame], ignore_index=True)

        #Wait two seconds before proceeding to the next Selenium action
        time.sleep(2)
        #Press Back Button
        driver.back()
        
        
        
        
    #Route will not be selected if not visible on screen, hence if ElementClickInterceptedException is brought up
    #the page is scrolled down slightly and code can proceed
    except ElementClickInterceptedException:
        driver.execute_script("window.scrollBy(0, 300);")
        
        #Part 1 - Entering ith Route 
        
        # Click on the ith Route
        Route_Stops[i].click()
        # Wait two seconds before proceeding to the next Selenium action
        time.sleep(2)
        # Obtaining Name of Route (Route XXX)
        Route_Name = driver.find_element(By.XPATH,'/html/body/form/div[3]/section[2]/div/div/div[1]/h1').text
        # Print Current Route Name to have an idea how for loop is progressing
        print(Route_Name)
        
        #Part 2 - Expaning to Obtain All Stops and all times

        # Expanding to See times only for stops (times for specific cities are hidden)
        
        # Removing Times for all cities (Done by changing Class to 'main-row first opened')
        Expand_All_Button1_XPATH = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[not(position()=1 or position()=2)]/div/table/tbody/tr[2]/td/div/table/tbody/tr[1]'
        ShowStops_Button1 = driver.find_elements(By.XPATH,Expand_All_Button1_XPATH)
        New_Class = 'main-row first opened'
        for i in range(len(ShowStops_Button1)):
            driver.execute_script("arguments[0].setAttribute('class', arguments[1]);", ShowStops_Button1[i], New_Class)

        # Obtaining Times for only Stops (Done by changing Class to 'main-row first opened')
        Expand_All_Button2_XPATH = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[not(position()=1 or position()=2)]/div/table/tbody/tr[2]/td/div/table/tbody/tr[not(position()=1)]'
        ShowStops_Button2 = driver.find_elements(By.XPATH,Expand_All_Button2_XPATH)
        New_Class = 'main-row additional opened'
        for i in range(len(ShowStops_Button2)):
            driver.execute_script("arguments[0].setAttribute('class', arguments[1]);", ShowStops_Button2[i], New_Class)

        # Show all times available for each stop throughout the entire day
        
        # Obtain all possible times (Done by changing class to 'schedule-part visible')
        Show_All_Time = driver.find_elements_by_class_name('schedule-part')
        new_class = "schedule-part visible"
        for i in range(len(Show_All_Time)):
            driver.execute_script("arguments[0].setAttribute('class', arguments[1]);", Show_All_Time[i], new_class)

        #Part 3 - Finding Route Directions
        Route_Direction = driver.find_elements(By.XPATH,'/html/body/form/div[3]/section[2]/div/div/div[2]/div/div/table/thead/tr/th')

        #Part 4 - for loop going over all different routes in Route_Direction list 
        # This is followed for loop going over all dates in the specific Route_Direction
        
        #first loop going over all different routes in Route_Direction list
        for A in range(len(Route_Direction)):
            
            DataFrames_Time = []
            
            # Obtaining Selenium Elements of all Dates in Ath Route Direction
            Dates_XPATH = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[1]/td[1]/select/option'
            Dates = driver.find_elements(By.XPATH,Dates_XPATH)
            
            
            #Second for loop going over all different dates in Ath Route_Direction 
            for C in range(len(Dates)):

                #Creating empty list to hold dataframes for Cities, Stops and corresponding Stops_Time
                Cities = []
                Stops = []
                Stops_Time = []

                #Click on C date 
                Dates[C].click()
                
                # Part 4.1 -  Calculate Number of Stop Times Per Stop
                #First Stop is used since all stops should have the same number of stop times
                Stop_Length_Xpath = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[2]/td/div[{C+1}]/table[1]/tbody/tr[1]/td/span'
                No_Stops_Per_Stop = driver.find_elements(By.XPATH,Stop_Length_Xpath)
                Length = len(No_Stops_Per_Stop)-1
                    
                
                # Part 4.2 - Collect All Cities in which bus stops are located  
                City_Xpath = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[2]/td/div[{C+1}]/table/tbody/tr[1]/td[1]/span'
                City_Name = driver.find_elements(By.XPATH, City_Xpath)
                for i in range(len(City_Name)):
                    Cities.append(City_Name[i].text)
                Cities = [item for item in Cities if item]
                
                # Part 4.3 - Collect All Bus Stop Locations
                Stop_Xpath = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[2]/td/div[{C+1}]/table/tbody/tr[not(position()=1)]/td[1]/span'
                Stop_Name = driver.find_elements(By.XPATH, Stop_Xpath)
                for i in range(len(Stop_Name)):
                    Stops.append(Stop_Name[i].text)
                Stops = [item for item in Stops if item]
                
                # Part 4.4 - Obtain Stop Times of All Bus Stop Location 
                # NB: Stop Times will also include Cities and Stops names  
                # All found in Cities and Stops list
                Times_Xpath = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[2]/td/div[{C+1}]/table/tbody/tr/td/span'
                Stop_Times = driver.find_elements(By.XPATH,Times_Xpath)
                for i in range(len(Stop_Times)):
                    Stops_Time.append(Stop_Times[i].text)
                Stops_Time = [item for item in Stops_Time if item]

                # Part 4.5 - Remove Cities and Stops from Stops_Time
                Stops_Time = [item for item in Stops_Time if item not in Cities]
                Stops_Time = [item for item in Stops_Time if item not in Stops]

                # Part 4.6 - Creating DataFrame consisting of one column for each time entry in a stop
                
                # Create a dictionary with L lists named 'Stop Time 1', 'Stop Time 2', etc.
                Stop_Times_Predf = {f'Stop Time {i + 1}': [] for i in range(Length)}
                # Distribute the items in the Stops_Time into Stop_Times_Predf
                for i, item in enumerate(Stops_Time):  # Assuming A is the list of items
                    Stop_Times_Predf[f'Stop Time {(i % Length) + 1}'].append(item)
                # Creating Dataframe using Stop_Times_Predf
                df = pd.DataFrame(Stop_Times_Predf)

                # Part 4.7 - Input Route Date, Stops and City Name into df
                
                # Insert Current Date to df
                df.insert(0, 'Date', Dates[C].text)
                # Insert list of Stops to df
                df.insert(0,'Stops', Stops)

                # Creating a Dataframe consisting of list of stops and their respective cities
                # Similar to vlookup in Excel, each stop will then be labbelled with a corresponding city
                
                #Obtain a list of selenium objects representing all cities and stops
                City_Stop_Xpath = f'/html/body/form/div[3]/section[2]/div/div/div[2]/div[{A+3}]/div/table/tbody/tr[2]/td/div[{C+1}]/table/tbody/tr/td[1]/span'
                City_Stop = driver.find_elements(By.XPATH, City_Stop_Xpath)
                City_Stop_List = []
                for i in range(len(City_Stop)):
                    City_Stop_List.append(City_Stop[i].text)
                
                data = []
                current_b_item = None
                #for loop going over list of City and Stops and if a city is found (done by comparing
                #City_Stop_List to Cities list it is appended to a list called current_b_item)
                for item in City_Stop_List:
                    if item in Cities:
                        current_b_item = item
                    else:
                        if current_b_item:
                            data.append([item, current_b_item])
                            
                # Creating City_Stop_DataFrame consisting of Stops and City Name Column
                City_Stop_DataFrame = pd.DataFrame(data, columns=['Stops', 'City Name'])
                # Remove duplicates from City_Stop_DataFrame to ensure 
                City_Stop_DataFrame.drop_duplicates(inplace = True)
                # Adding index to df to ensure it is not shuffled after carrying out merge function 
                df['original_index'] = df.index
                # Ensure both 'Stops' columns in df and City_Stop_DataFrame are strings
                City_Stop_DataFrame['Stops'] = City_Stop_DataFrame['Stops'].astype(str)
                df['Stops'] = df['Stops'].astype(str)
                # Using pd.merge to carry out operation similar to VLOOKUP-like operation
                DataFrame_OneTime = pd.merge(City_Stop_DataFrame, df, on='Stops', how='left')
                # Sort back to the original order using the index and re-name dataframe to DataFrame_OneTime
                DataFrame_OneTime = DataFrame_OneTime.sort_values('original_index').drop('original_index', axis=1)
                
                #Part 4.8 - Append DataFrame_OneTime to DataFrames_Time
                DataFrames_Time.append(DataFrame_OneTime)
                
                #Part 4.9 - Append Dataframe for all Times
                OneRoute_AllTimes_DataFrame = pd.concat(DataFrames_Time, ignore_index=True)

            # Part 4.10 - Append Ath Route_Direction to OneRoute_AllTimes_DataFrame and concatenate to 
            # Final_Dataframe which will compile all Dataframes
            OneRoute_AllTimes_DataFrame.insert(0,'Route Direction',Route_Direction[A].text)
            Final_Dataframe = pd.concat([Final_Dataframe, OneRoute_AllTimes_DataFrame], ignore_index=True)

        #Wait two seconds before proceeding to the next Selenium action
        time.sleep(2)
        #Press Back Button
        driver.back()

Route X1


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=129.0.6668.70)


In [ ]:
#Split Route Direction Column into Route Direction and Route Number
Final_Dataframe[['Route Direction', 'Route Number']] = Final_Dataframe['Route Direction'].str.split('\n', expand=True)
# Move Route Number Column from last to first column
Last_Column = Final_Dataframe.pop(Final_Dataframe.columns[-1])
# Insert Last_Column as First Column
Final_Dataframe.insert(0, Last_Column.name, Last_Column)

In [ ]:
#Saving Final Dataframe
Final_Dataframe.to_csv('C:\\Users\\Owner\\ICT5012 - Disseration\\All_Routes.csv', index=False)  